## Maps of pages
Mappiamo le pagine a disposizione e le loro caratteristiche. Ogni blocco di codice prova a rispondere a una domanda.

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd   
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt

interactions = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/interacions/facebook_outreach_vs_interaction.csv', usecols=['post_id', 'alpha', 'outreach'])
posts = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_posts.csv', usecols=['created_time', 'page_id', 'post_id'], encoding='ISO-8859-1')

posts_dict = posts.set_index('post_id').T.to_dict()

interactions['created_time'] = interactions['post_id'].map(lambda x: posts_dict.get(x, {}).get('created_time', None))
interactions['page_id'] = interactions['post_id'].map(lambda x: posts_dict.get(x, {}).get('page_id', None))

interactions=interactions.dropna()
print(interactions.head())

/home/jacoponudo/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/tmp/ipykernel_20381/577968565.py:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  posts_dict = posts.set_index('post_id').T.to_dict()


                            post_id  alpha   outreach  \
1     35281584398_10151655784439399   True   1350.125   
3   113349742029506_644708212226987   True      6.500   
4      7155422274_10153792989482275   True  44599.625   
8       67175109350_205981829418171   True     43.125   
11     7155422274_10151218230287275   True  15919.625   

                created_time          page_id  
1   2013-06-10T16:04:00+0000      35281584398  
3   2013-09-30T06:45:27+0000  113349742029506  
4   2015-10-07T19:44:42+0000       7155422274  
8   2011-03-01T17:33:52+0000      67175109350  
11  2012-08-24T08:15:30+0000       7155422274  


Quante pagine ci sono? 

In [2]:
len(interactions['page_id'].unique())

92

Di quante abbiamo la serie storica dei dati di attività dal 2010 al 2015?

In [3]:
outreach = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_outreachs.csv')
pages = outreach.groupby('page_id').size().reset_index(name='TS_coverage(W)')

pagine_con_oltre_200_righe = (pages['TS_coverage(W)'] > 48).sum()
print(f"Pagine con più di 200 righe: {pagine_con_oltre_200_righe}")

Pagine con più di 200 righe: 88


Dividiamo l'outreach iniziale in 3 classi bilanciate.

In [13]:
outreach = pd.read_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_outreach.csv')
outreach.columns=['Date','outreach','page_id']
import pandas as pd

# Filtra le pagine con TS_coverage(W) > 48 e ottieni i page_id
focus = pages[pages['TS_coverage(W)'] > 48]['page_id']

# Filtra le interazioni basandosi sui page_id di focus
first_outreachs = outreach[outreach['page_id'].isin(focus)].sort_values(by='Date').groupby('page_id').head(4)['outreach'].mean().reset_index()


AttributeError: 'numpy.float64' object has no attribute 'reset_index'

In [8]:
import pandas as pd

# Filtra le pagine con TS_coverage(W) > 48 e ottieni i page_id
focus = pages[pages['TS_coverage(W)'] > 48]['page_id']

# Filtra le interazioni basandosi sui page_id di focus
outreach = outreach[outreach['page_id'].isin(focus)]

# Ordina le interazioni in base a created_time
outreach = outreach.sort_values(by='created_time')

# Seleziona solo le prime 100 righe in ordine cronologico per ogni page_id
forth_outreachs= outreach.sort_values('created_time').groupby('page_id').head(4)

# Calcola la media di outreach per ogni page_id
first_outreach = forth_outreachs.groupby('page_id')['outreach'].mean().reset_index()

# Crea le classi omogenee per l'outreach usando i quantili
quantiles = first_outreach['outreach'].quantile([0, 0.25, 0.5, 0.75, 1]).values
labels = ['Molto Basso', 'Basso', 'Medio', 'Alto']
first_outreach['outreach_binned'] = pd.cut(first_outreach['outreach'], bins=quantiles, labels=labels, include_lowest=True)

# Mantieni solo le righe uniche nel DataFrame 'first_outreach' per il merge
first_outreach_unique = first_outreach[['page_id', 'outreach_binned','outreach']].drop_duplicates('page_id')

# Aggiungi la categoria al DataFrame 'pages' usando un merge
pages = pages.merge(first_outreach_unique, on='page_id', how='left')

# Stampa i limiti delle classi
for label, lower, upper in zip(labels, quantiles[:-1], quantiles[1:]):
    print(f"Classe: {label}, Estremo inferiore: {lower:.2f}, Estremo superiore: {upper:.2f}")

# Rinomina le colonne di 'pages' se necessario
pages.columns = ['page_id', 'TS_coverage(W)', 'initial_class_size', 'initial_size']


Classe: Molto Basso, Estremo inferiore: 0.00, Estremo superiore: 4.60
Classe: Basso, Estremo inferiore: 4.60, Estremo superiore: 9.18
Classe: Medio, Estremo inferiore: 9.18, Estremo superiore: 19.84
Classe: Alto, Estremo inferiore: 19.84, Estremo superiore: 12147.38


Dividiamo l'outreach finale in 3 classi bilanciate.

In [10]:
# Seleziona solo le prime 100 righe in ordine cronologico per ogni page_id
top_interactions = interactions.sort_values('created_time').groupby('page_id').tail(100)

# Calcola la media di outreach per ogni page_id
last_outreach = top_interactions.groupby('page_id')['outreach'].mean().reset_index()

# Crea le classi omogenee per l'outreach usando i quantili
quantiles = last_outreach['outreach'].quantile([0, 1/4, 1/2, 3/4, 1]).values
labels = ['Molto Basso', 'Basso', 'Medio', 'Alto']
last_outreach['outreach_binned'] = pd.cut(last_outreach['outreach'], bins=quantiles, labels=labels, include_lowest=True)

# Mantieni solo le righe uniche nel DataFrame 'last_outreach' per il merge
last_outreach_unique = last_outreach[['page_id', 'outreach_binned', 'outreach']].drop_duplicates('page_id')

# Aggiungi la categoria al DataFrame 'pages' usando un merge
pages = pages.merge(last_outreach_unique, on='page_id', how='left')

# Stampa i limiti delle classi
for label, lower, upper in zip(labels, quantiles[:-1], quantiles[1:]):
    print(f"Classe: {label}, Estremo inferiore: {lower:.2f}, Estremo superiore: {upper:.2f}")

# Aggiorna i nomi delle colonne nel DataFrame 'pages'
pages.columns = ['page_id', 'TS_coverage(W)', 'initial_class_size', 'initial_size', 'last_class_size', 'last_size']


Classe: Molto Basso, Estremo inferiore: 0.00, Estremo superiore: 12.94
Classe: Basso, Estremo inferiore: 12.94, Estremo superiore: 159.29
Classe: Medio, Estremo inferiore: 159.29, Estremo superiore: 3163.12
Classe: Alto, Estremo inferiore: 3163.12, Estremo superiore: 428301.75


Qual'è il delta della crescita tra prima e dopo.

In [11]:
pages['delta_growth']=pages['last_size']-pages['initial_size']

In quale arco temporale lavorano.

In [12]:
interactions['created_time'] = pd.to_datetime(interactions['created_time'])
diff_days = interactions.groupby('page_id')['created_time'].agg(max_date='max', min_date='min')
diff_days['diff_days'] = (diff_days['max_date'] - diff_days['min_date']).dt.days
pages = pages.merge(diff_days[['diff_days']], on='page_id', how='left')
pages['growth_rate']=pages['delta_growth']/pages['diff_days']

Esporta le pagine con le loro caratteristiche.

In [18]:
pages.to_csv('/home/jacoponudo/Documents/Size_effects/DATA/sample_pages.csv')